In [7]:
import requests
import hashlib
import time
from pprint import pprint

ZHENLV_URL = "http://api.tdxinfo.com/service/flight/domestic"
ZHENLV_URL = "http://apis.travelzen.com/service/flight/domestic"
ZHENLV_USER = "563716d245ced1d2ea3ffe61"
ZHENLV_SIGN_CODE = "3e62869b"
ZHENLV_RAW = '<?xml version="1.0" encoding="UTF-8" standalone="yes"?><TzRequest xmlns="http://www.travelzen.com/flight/base" xmlns:domestic="http://www.travelzen.com/flight/domestic" xmlns:insure="http://www.travelzen.com/flight/insure" xmlns:international="http://www.travelzen.com/flight/international" xmlns:basic="http://www.travelzen.com/flight/basic" ><requestMetaInfo><userName>{user}</userName><signCode>{sign}</signCode><timeStamp>{ts}</timeStamp><responseDataType>JSON</responseDataType></requestMetaInfo><requestEntity xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:type="domestic:FlightSearchV2Request"><domestic:flightRange><domestic:fromCity>{from_city}</domestic:fromCity><domestic:toCity>{to_city}</domestic:toCity><domestic:fromDate>{date}</domestic:fromDate></domestic:flightRange><domestic:lowestPrice>false</domestic:lowestPrice><domestic:openingTime>true</domestic:openingTime></requestEntity></TzRequest>'

from_city="SHA"
to_city="BJS"
date="2018-03-02"
line = "{}-{}".format(from_city, to_city)
route = "{}-{}-{}".format(from_city, to_city, date)
calendar_price = 0
time_stamp = int(time.time())
hstr = "tzOpenapisignCode{}timeStamp{}userName{}tzOpenapi".format(ZHENLV_SIGN_CODE, time_stamp, ZHENLV_USER)
sign = hashlib.md5(hstr.encode('utf-8')).hexdigest()
url = ZHENLV_URL
raw = ZHENLV_RAW.format(user=ZHENLV_USER,
                             sign=sign,
                             ts=time_stamp,
                             from_city=from_city,
                             to_city=to_city,
                             date=date)

resp = requests.post(
    url,
    data=raw,
    headers={"Content-Type": "application/xml"}
)
resp_data = resp.json()


In [11]:
data = resp_data['FlightSearchV2Response']['flightSegmentResult'][0]

flight_objects = data.get('flightObject', [])
for flight_object in flight_objects:
    lowest_price = 0
    cabin_infos = flight_object.get('cabinInfo', [])
    for cabin_info in cabin_infos:
        if not lowest_price:
            lowest_price = cabin_info.get('facePrice', 0)
        elif 0 < cabin_info.get('facePrice', 0) < lowest_price:
            lowest_price = cabin_info['facePrice']

    if lowest_price and not calendar_price:
        calendar_price = lowest_price
    elif 0 < lowest_price < calendar_price:
        calendar_price = lowest_price

route,calendar_price

('SHA-BJS-2018-03-02', 530)